In [ ]:
# Add noise to the simulated light curves
# By add noise from Kepler Dataset
# Here entire std dev is considered
# Kepler_noise_addn_v1.ipynb


# In other versions more refined std_dev will be considered

In [ ]:
# Import TF and check for GPU

import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))
print("TensorFlow version:", tf.__version__)

# Import required libraries

import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
from keras import layers
from tensorflow.keras.models import save_model, load_model
import math
from sklearn.model_selection import train_test_split
import sys
import random 


In [ ]:
# 1. Load Dataset
## Load Train Set
train_shape_dir = '/home/abraham/Documents/ms_proj_shape_lc_gen/data_npy/shape_npy/shape_filled5.npy'
train_lc_dir = '/home/abraham/Documents/ms_proj_shape_lc_gen/data_npy/lc_npy/lc_dict_5.npy'
train_lc = np.load(train_lc_dir)
train_shape = np.load(train_shape_dir)
# Check equality of number of dataset
if len(train_lc)==len(train_shape):
    print("Train Set: No. of LC = No. of shapes")
else:
    sys.exit("EXIT: Train Set: No. of LC != No. of shapes")

## Load Validation Set
vald_shape_dir = '/home/abraham/Documents/ms_proj_shape_lc_gen/data_npy/shape_npy/shape_filled4.npy'
vald_lc_dir = '/home/abraham/Documents/ms_proj_shape_lc_gen/data_npy/lc_npy/lc_dict_4.npy'
vald_lc = np.load(vald_lc_dir)
vald_shape = np.load(vald_shape_dir)
# Check equality of number of dataset
if len(vald_lc)==len(vald_shape):
    print("Vald Set: No. of LC = No. of shapes")
else:
    sys.exit("Vald Set: No. of LC = No. of shapes")

In [ ]:
# 2. Normalize the image, convert to opacity map
## Train Set
train_shape = train_shape/np.amax(train_shape)
train_shape_where_0 = np.where(train_shape == 0)
train_shape_where_1 = np.where(train_shape == 1)
train_shape[train_shape_where_0] = 1  # 1 represent the shape (1 opacity)
train_shape[train_shape_where_1] = 0  # 0 represent background (0 opacity)

## Valdn Set
vald_shape = vald_shape/np.amax(vald_shape)
vald_shape_where_0 = np.where(vald_shape == 0)
vald_shape_where_1 = np.where(vald_shape == 1)
vald_shape[vald_shape_where_0] = 1  # 1 represent the shape (1 opacity)
vald_shape[vald_shape_where_1] = 0  # 0 represent background (0 opacity)
print("Normalized the shape")


In [ ]:
# 2. Normalize the lightcurves
## - Train Set
train_lc_scaled = np.zeros(train_lc.shape)
for i in np.arange(len(train_lc_scaled)):
    train_lc_scaled[i] = (train_lc[i] - np.amin(train_lc[i]))/(np.amax(train_lc[i]) - np.amin(train_lc[i]))

## - Vald Set
vald_lc_scaled = np.zeros(vald_lc.shape)
for i in np.arange(len(vald_lc_scaled)):
    vald_lc_scaled[i] = (vald_lc[i] - np.amin(vald_lc[i]))/(np.amax(vald_lc[i]) - np.amin(vald_lc[i]))
print("Normalized the light curves")


In [ ]:
# Add flat line towards left and right of dip
# 50 data points on each side
# 3. Extend the lightcurves
## - Train Set
train_lc_scaled_append = np.ones((train_lc.shape[0],120))
print('train_lc_scaled_append.shape = ',train_lc_scaled_append.shape)
print("len(train_lc_scaled_append[0,10:110]) = ",len(train_lc_scaled_append[0,10:110]))

for i in np.arange(len(train_lc_scaled)):
    train_lc_scaled_append[i,10:110] = train_lc_scaled[i]

## - Vald Set
vald_lc_scaled_append = np.ones((vald_lc.shape[0],120))
for i in np.arange(len(vald_lc_scaled)):
    vald_lc_scaled_append[i,10:110] = vald_lc_scaled[i]
print("Extended the light curves")

Don't run above code more than once anymore

In [ ]:
# Plot - Train LCs
num = 5
fig,ax=plt.subplots(num,2, figsize=(5,8), gridspec_kw={ 'width_ratios': [2,1],
        'wspace': 0.2,'hspace': 0.4})

ax[0][1].set_title('Shape',size=15)
ax[0][0].set_title('Light Curve (Train Dataset)',size=15)
ax[num-1][0].set_xlabel('Phase',size=13)
ph = np.linspace(-1,1,len(train_lc_scaled_append[0]))
# advance = 60

i = 0
for i in np.arange(0,num):
    k = random.randint(0, len(train_lc_scaled_append)-1)
    ax[i][1].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
    if(i<num-1): ax[i][0].tick_params(labelbottom = False, bottom = False)
    img = ax[i][1].imshow(train_shape[k],cmap='inferno')
    plt.colorbar(img)
    ax[i][0].set_ylabel('Flux',size=13)
    ax[i][0].set_ylim(-0.5,1.5)
#     ax[i][0].scatter(ph, vald_lc_scaled_append[k],color = 'black',marker='.')
    ax[i][0].plot(ph, train_lc_scaled_append[k],color = 'black',linewidth='3')
    ax[i][0].grid('on')
    i = i + 1

In [ ]:
filenames = ['/home/abraham/Documents/tfr_shallue/tfr_all/tfrecord/train-00000-of-00008',
             '/home/abraham/Documents/tfr_shallue/tfr_all/tfrecord/train-00001-of-00008',
             '/home/abraham/Documents/tfr_shallue/tfr_all/tfrecord/train-00002-of-00008',
             '/home/abraham/Documents/tfr_shallue/tfr_all/tfrecord/train-00003-of-00008',
             '/home/abraham/Documents/tfr_shallue/tfr_all/tfrecord/train-00004-of-00008',
             '/home/abraham/Documents/tfr_shallue/tfr_all/tfrecord/train-00005-of-00008',
             '/home/abraham/Documents/tfr_shallue/tfr_all/tfrecord/train-00006-of-00008',
             '/home/abraham/Documents/tfr_shallue/tfr_all/tfrecord/train-00007-of-00008',
             '/home/abraham/Documents/tfr_shallue/tfr_all/tfrecord/val-00000-of-00001',
             '/home/abraham/Documents/tfr_shallue/tfr_all/tfrecord/test-00000-of-00001']
raw_dataset = tf.data.TFRecordDataset(filenames)
raw_dataset

In [ ]:
feature_description = {
    'global_view': tf.io.FixedLenSequenceFeature([], tf.float32, default_value=0.0,allow_missing=True),
    'local_view': tf.io.FixedLenSequenceFeature([], tf.float32, default_value=0.0,allow_missing=True),
    'av_training_set': tf.io.FixedLenFeature([], tf.string, default_value=''),
    'kepid': tf.io.FixedLenSequenceFeature([], tf.int64, default_value=int(0),allow_missing=True),
}

def _parse_function(example_proto):
  # Parse the input `tf.train.Example` proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, feature_description)

# At this point the dataset contains serialized tf.train.Example messages.
# When iterated over it returns these as scalar string tensors.
# Use the .take method to only show the first 10 records.
for raw_record in raw_dataset.take(10):
  print(repr(raw_record))
  
parsed_dataset = raw_dataset.map(_parse_function)
parsed_dataset


In [ ]:
# Calculate the size of one light curve corresponding to 'local_view'
y = np.array([])
for elem in parsed_dataset.take(1):
  y = np.append(y,[elem['local_view']])
length_lc = len(y)
print("length_lc = " ,length_lc)


# Calculate total number of light curves
no_data = 0
for elem in parsed_dataset.as_numpy_iterator():
    no_data = no_data + 1
print('no_data = ',no_data)

In [ ]:
# Extract Kepler ID

kepid_array = np.zeros(shape=(no_data,))
i = 0
for elem in parsed_dataset:
    kepid_array[i] = elem['kepid'] 
    i = i + 1

In [ ]:
print(kepid_array[908:912])

In [ ]:
# convert all tf dataset to np array (light curve numpy array (lc_np_array))
lc_np_array = np.zeros(shape=(no_data,length_lc))
lc_np_array.shape


i = 0
for elem in parsed_dataset.as_numpy_iterator():
    lc_np_array[i] = elem['local_view'] #+1.0
    i = i + 1

print('lc_np_array[0] = ',lc_np_array[0])


In [ ]:
for i in np.arange(1):
    i=8970
    test_lc = lc_np_array[i]

    ph = np.linspace(-1.0,1.0,len(lc_np_array[0]))
    print('len(ph) = ',len(ph))
    plt.scatter(ph[0:60], test_lc[0:60])
    print('len(ph[0:60]) = ',len(ph[0:60]))
    plt.scatter(ph[141:202], test_lc[141:202])
    # print('len(ph[141:202]) = ',len(ph[141:202]))
    plt.scatter(ph[60:141], test_lc[60:141])

    plt.show()

In [ ]:
# Noise Array
# Append first 60 and last 60 elements
noise_array = np.zeros((len(lc_np_array),120))
for i in np.arange(len(lc_np_array)):
    noise_array[i][0:60] = lc_np_array[i,0:60]
    noise_array[i][60:120] = lc_np_array[i,141:202]


ph = np.linspace(-1.0,1.0,len(noise_array[0]))
plt.scatter(ph, noise_array[0])
print('len(ph) = ',len(ph))
print('noise_array.shape = ',noise_array.shape)

plt.show()

In [ ]:
# Calc std dev of the arrays
std_devs_Kepler = np.array([np.std(arr) for arr in noise_array])

# Plot histogram of std dev
# bins = np.logspace(-5,5,100)
# bins = np.linspace(0,10**2,11)
bins = np.linspace(0,2,100)

plt.figure(figsize=(9, 9))
# plt.hist(std_devs_Kepler, bins=bins, density=True, alpha=0.8, color='tab:green')
a,*_ = np.histogram(std_devs_Kepler, bins=bins)
print('a = ',a)
a_percent = (a/np.sum(a))*100
print('a_percent = ',a_percent)

plt.stairs(a_percent, bins, baseline=0,fill=True,color='black')
plt.xlabel('Sigma')
# plt.xscale("log")
plt.ylabel('Probability (%)')
plt.title('Histogram - Standard Deviation')
plt.grid(True)
plt.show()

# Add noise to train lightcurve
#

In [ ]:
# Add noise to simulated train light curve, such that index is uniform
index_noise_train= np.random.uniform(0,len(noise_array),(len(train_lc_scaled_append),))
train_lc_scaled_append_noise = np.zeros(train_lc_scaled_append.shape)
print("index_noise_train.shape = ",index_noise_train.shape)
i = 0
for i in np.arange(len(train_lc_scaled_append)):
    index = int(index_noise_train[i])
    print(index)
    train_lc_scaled_append_noise[i] = train_lc_scaled_append[i] + noise_array[index]


In [ ]:
# Plot - Train LCs
num = 5
fig,ax=plt.subplots(num,4, figsize=(10,10), gridspec_kw={ 'width_ratios': [2,3,3,3],
        'wspace': 0.4,'hspace': 0.7})

ph = np.linspace(-1,1,len(train_lc_scaled_append_noise[0]))
ph_kepler = np.linspace(-1,1,len(lc_np_array[0]))

i = 0
for i in np.arange(0,num):
    k = random.randint(0, len(train_lc_scaled_append_noise)-1)
    original_kelper_index = int(index_noise_train[int(k)])


    ax[i][0].set_title('Shape',size=13)
    img = ax[i][0].imshow(train_shape[k],cmap='inferno')
    plt.colorbar(img)

    ax[i][1].set_title('Simulated LC',size=13)
    ax[i][1].plot(ph, train_lc_scaled_append[k],color = 'black',linewidth='3')

    ax[i][2].set_title(f'Kepler LC. std_dev = {np.round(std_devs_Kepler[original_kelper_index],4)}',size=13)
    ax[i][2].plot(ph_kepler, lc_np_array[original_kelper_index]+1,color = 'black',linewidth='3')
    
    ax[i][3].set_title('Noise Added LC',size=13)
    ax[i][3].plot(ph, train_lc_scaled_append_noise[k],color = 'black',linewidth='3')

plt.suptitle("Train Set")

plt.show()


# Adding noise to Validation Dataset

In [ ]:
# Add noise to simulated vald light curve, such that index is uniform
index_noise_vald= np.random.uniform(0,len(noise_array),(len(vald_lc_scaled_append),))
vald_lc_scaled_append_noise = np.zeros(vald_lc_scaled_append.shape)
print("index_noise_vald.shape = ",index_noise_vald.shape)
i = 0
for i in np.arange(len(vald_lc_scaled_append)):
    index = int(index_noise_vald[i])
    print(index)
    vald_lc_scaled_append_noise[i] = vald_lc_scaled_append[i] + noise_array[index]

In [ ]:
# Plot - Vald LCs
num = 5
fig,ax=plt.subplots(num,4, figsize=(10,10), gridspec_kw={ 'width_ratios': [2,3,3,3],
        'wspace': 0.4,'hspace': 0.7})

ph = np.linspace(-1,1,len(vald_lc_scaled_append_noise[0]))
ph_kepler = np.linspace(-1,1,len(lc_np_array[0]))

i = 0
for i in np.arange(0,num):
    k = random.randint(0, len(vald_lc_scaled_append_noise)-1)
    original_kelper_index = int(index_noise_vald[int(k)])


    ax[i][0].set_title('Shape',size=13)
    img = ax[i][0].imshow(vald_shape[k],cmap='inferno')
    plt.colorbar(img)

    ax[i][1].set_title('Simulated LC',size=13)
    ax[i][1].plot(ph, vald_lc_scaled_append[k],color = 'black',linewidth='3')

    ax[i][2].set_title(f'Kepler LC. std_dev = {np.round(std_devs_Kepler[original_kelper_index],4)}',size=13)
    ax[i][2].plot(ph_kepler, lc_np_array[original_kelper_index]+1,color = 'black',linewidth='3')
    
    ax[i][3].set_title('Noise Added LC',size=13)
    ax[i][3].plot(ph, vald_lc_scaled_append_noise[k],color = 'black',linewidth='3')

plt.suptitle("Validation Set")
plt.show()
